# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/miniconda3/envs/yeqing/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/yeqing/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/root/miniconda3/envs/yeqing/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是八卦之一，也是八宫图说、易经、易学中的重要元素。乾卦代表天、强、刚、动、健等含义。乾卦是由两个阴爻夹一个阳爻构成，象征天、强大、坚毅、刚健等品质。

乾卦的卦辞是：“天行健，君子以自强不息。”这意味着，乾卦象征着天的行健，即不断前进、自强不息的品质。君子应该效仿乾卦的这种品质，不断地自强、奋斗，以达到自己的目标。

乾卦的象征意义非常丰富，可以用来象征君主、领导、权力、刚强、运动等。在易经中，乾卦的排列可以是天、地、风、雨、云、雷、山、泽等，这些排列组合可以形成不同的象征意义。乾卦的组合也可以与其他卦相组合，形成更加复杂的象征意义。

乾卦的启示是，我们应该效仿乾卦的品质，自强不息，不断前进，以达到自己的目标。同时，我们也应该理解乾卦的象征意义，并将其应用到自己的生活中，以获得更多的启示和帮助。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，也是八宫图说、易经、易学中的重要元素。讼卦象征诉讼、纷争、争斗等。它由两个阴爻夹一个阳爻构成，象征女性（阴）在争斗中处于劣势，需要男性的帮助才能获得平局。因此，讼卦也象征着男女之间、家庭之间、朋友之间、国家与国家之间的矛盾和纷争，需要通过和平、公正、公平的方式来解决。

更多关于讼卦的卦辞和解释，请参考下述示例：

《易经》中，讼卦的卦辞是：“天象地的，君子以明 marked 。” 这意味着，讼卦象征着天象地的情况，即天地自然规律的明显表现。君子应该效仿这种表现，明确自己的责任和义务，以维护社会秩序和公平正义。

在八宫图说中，讼卦位于北方，与事业、努力、艰难、挑战、努力等有关。这意味着，诉讼和纷争往往来自于努力和艰难的过程中，需要付出更多的努力和耐心才能解决。

在易学中，讼卦的启示是，我们应该明确自己的责任和义务，以避免争斗和纷争。同时，我们也应该尊重自然规律和社会公正，努力维护和平、公正、公平的社会环境。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20250817_002513"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦，是周易中的一卦，由两个乾卦叠加而成，代表天。它象征着宇宙的运行和自然的法则。在这个卦中，预示着大吉大利，预示着君子应顺从天道，勇敢地前进，顺利地完成使命。', 'content': '\n乾卦的核心哲学是：“天行健，君子健 too”（天行健，君子健 also）。这意味着，天行健，君子应当效仿天的行健，即坚强地运行。虽然君子位居于卦的上面，但也要像天一样坚强地运行，才能顺利解决困难，摆脱困境。\n\n乾卦的卦象是：下卦为天，上卦为乾。天行健，君子健也。乾卦象征着天行健，君子应当效仿天的行健，坚强地运行。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250817_002513）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250817_002513）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象，它由上卦乾（天）和下卦坎（水）组成。坎象征着详和的鳄鱼，而乾象征着天。这个卦象预示着详和的鳄鱼之水, 但也提醒了天象的险恶。因此，这是一个充满警示的卦象。

在这个卦象中，天雷震卦与坎卦相互交织，形成了一个复杂的环境。雷震象征着刚毅坚定，但也意味着刚毅坚定会引发 conflict。因此，这个卦象告诉人们，如果能够坚持原则，但如果过于刚硬，则容易引发冲突。

讼卦的核心哲学是：矛盾纷争激烈，但详和的鳄鱼能化解矛盾，提醒人们 to resolve conflicts through harmony and cooperation. It is a call to action to resolve disputes through compromise and understanding, rather than through force or aggression.

因此，讼卦鼓励人们以理智和善意来化解纷争，强调通过合作与沟通，可以化解矛盾，最终得到圆满的结果。


In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250817_002513）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [15]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。它反映了天地间阳刚之气的存在，以及天地之间相互信任和相互依赖的关系。

在卜辞中，乾卦预示着天象的运行，以及人类对天象的依赖。同时，它也预示着人类应当刚健强健，积极进取，勇往直前。

在解卦时，取刚健强健之象，象征天之无私，刚健强劲，无所畏惧。在meta-comment中，它提示人们刚健强健，勇往直前，无所畏惧，代表刚健强劲的特性。

在推卦时，则强调刚健强健，柔中带刚，刚中带柔，提示人们刚健强健，勇往直前，无所畏惧。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。它预示着刚健强健的形势，同时也提醒要小心谨慎。根据《象辞》，乾卦象征着刚健强健的特性，同时也要小心谨慎。根据《象辞》，乾卦的卦辞为：“乾，元亨利贞”，这表示它代表天，刚健强健的特性，同时也象征刚健强健的形势。

乾卦表示天，天是刚健强健的，刚强而运行不已。这刚健強健的形势是您所面临的挑战，也是您用来行动的力。根据《辞》，乾卦所象征的刚健强健的特性是：刚健、坚贞、运行不已，这是您所面临挑战时必须具备的品质。

根据《易经》，乾卦的形势是刚健强健的，而所代表的运势是刚健强健的。这表示您在生活中的各种挑战中，需要以刚健强健的毅力来应对，并坚持到底。

总结起来，乾卦是周易中的一卦，代表天，预示着刚健强健的形势，同时也提醒要小心谨慎。它提醒您，在生活中的各种挑战中，要以刚健强健的毅力来应对，并坚持到底。


In [16]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是周易中的一卦，对应着周易的卦辞是“地水师卦：地水（卦）为师（卦）”。这卦的主要含义是，尽管看起来情况并不顺利，但事实上，这是一段美好的历程，因为尽管身处艰难的环境中，但却能保持镇静，并得到周围人的支持。

在占卜中，地水师卦表示事情开始时可能会遇到一些艰难困苦的情况，周围的环境并不顺利，会给人带来压力和挑战。然而，只要准备充分，保持冷静，积极应对，就能化险为夷，达到预期目的。

在解卦时，地形水利，说明顺利，可以得到周围人的帮助。在决策时，应该考虑到事情的实际情况，分析形势，做出明智的选择。

地水师卦的分析需要结合具体的时代背景和情境，不能简单地套用。在不同的情况下， interpreter 可能会给出不同的解释。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 地水师卦是周易中的一卦，由上卦坤（地）和下卦坎（水）组成，上卦为地，下卦为水，象征着地中有水，地中有生命的成长过程。根据《象辞》，这一卦象被解释为“地水”，具有“水国”之象。

根据《象辞》，这个卦象表示的是"地中有水"，具有水国之象。这预示了好运 will be reached, and the safe and sound condition will be maintained. According to the commentators in the Book of Divination, the best course of action is to be patient and composed, and to avoid argumentative and confrontational situations.

地水师卦的卦象是地水，这预示了好运 will be reached, and the safe and sound condition will be maintained. According to the commentators in the Book of Divination, the best course of action is to be patient and composed, and to avoid argumentativ

In [17]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是周易中的一卦，对应着《易经》中的卦象为乾上坎下。这一卦的核心卦辞是“利见大人”，意味着有利可图，有君子之道。讼卦的核心意义是解决争端，君子应该明辨是非，善于用礼制来平息争端。在卜卦时，利于形成儒林之象，有利于建立人群之间相互协作的关系。

讼卦的哲学基础是：君子应该明辨是非，善用礼制平息争端。在处理争端时，应当时刻保持冷静，注意控制自己的情绪，避免过度冲动。通过冷静分析，找出问题的根源，从而找到解决问题的方法。

讼卦给我们的启示是：在面对争端和冲突时，我们应该保持冷静和理智，通过分析问题，找出根源，从而找到解决问题的方法。同时，我们还需要善于用礼制来平息争端，建立人群之间相互协作的关系。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么卦

 讼卦是周易中的一卦，代表诉讼、争斗和诉讼的卦象。它由两个卦象组成，分别为上卦坎（水）和下卦乾（天）。上卦为坎，象征水，表示上卦的特性为险，下卦为乾，象征天，表示下卦的特性为刚。双方相互克制，双方刚强，难以达成共识，因此双方需要斗争，这表示对方刚强，需要刚直之态度。讼卦的运势为吉，但需要刚健刚强，不能慈慈柔顺，否则容易对方刚强刚直，导致诉讼胜利的结论。

讼卦的取义是：水得刚肯，能平能和。天有刚健，不能容留。双方相互克制，需要刚直之态度。

讼卦的启示是：遇到困难和阻力，需要刚健刚强，坚持到底，不要慈慈柔顺，else 刚强刚直，导致诉讼胜利的结论。

讼卦的适宜读音是：讼卦，卦名：坎上乾下，象征天。

讼卦的运势是：凶，需要刚健刚强，不能慈慈柔顺，else 刚强刚直，导致诉讼胜利的结论。

讼卦的解卦是：坤上卦，代表地，象征柔顺。解卦是讼卦的解决方案，双方需要刚健刚强，不能慈慈柔顺，else 刚强刚直，双方容易达成共识，对方也容易退让，因此需要对方，对方容易退让，对方容易退让，对方容易退让。

讼卦的退卦是：乾上卦，代表天，象征刚直。退卦是讼卦的解决方案，对方容易退让，需要对方，对方容易退让，对方容易退让。

讼卦的象数是：天雷震卦（刚直）+ 坎水卦（险）= 刚健刚强

讼卦的运势总结是：对方刚强，需要刚直之态度，否则容易对方刚强刚直，导致诉讼胜利的结论。
